In [1]:
# 构建分类器 1.准备数据集

from sklearn.datasets import fetch_20newsgroups                          # 机器学习工具包下的数据集模块中的20新闻组模块

# dataset=fetch_20newsgroups(subset='all')    # 初始化一个新闻组提取器 subset='all' 取子集的所有数据（all所有 train训练 test测试）
# 运行会自动下载第二个版本20news-bydate.tar.gz  （非常卡  程序会崩溃）

# 提取部分类别（取5个类别）
sample_cate = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med', 'rec.sport.baseball']

# 取训练数据
train_set = fetch_20newsgroups(subset='train',categories=sample_cate,           # 初始化一个新闻组提取器
                               shuffle=True, random_state=42,
                               remove = ('headers','footers','quotes'))
# 取测试数据
test_set = fetch_20newsgroups(subset='test', categories=sample_cate,
                              shuffle=True, random_state=42,
                              remove = ('headers','footers','quotes'))

# subset='train' 取子集的训练数据   categories 设置提取的类别  
# shuffle=True:打乱数据            random_state=42 相当于一个随机种子(固定随机数) 
# remove=移除邮件的头部，尾部，引用

# print(train_set)                                          # 查询所有数据
print(len(train_set.data), len(test_set.data))            # 计算邮件内容数量            训练数据2854封邮件 测试数据1899封邮件
print(train_set.target_names)                             # 打印类别中的所有类别名       target_names是一个键 它里面包含所有类别的名字
print(train_set.target)                                   # 打印标签

2854 1899
['alt.atheism', 'comp.graphics', 'rec.sport.baseball', 'sci.med', 'soc.religion.christian']
[3 3 2 ... 1 4 1]


In [6]:
# 特征提取
from sklearn.feature_extraction.text import TfidfVectorizer            # tf-idf（词频计算方法）文本特征提取模块
tfv = TfidfVectorizer(stop_words='english')                  # 初始化一个tf-idf文本特征提取器 设置停用词排除不想要的特征文本
train_tfv = tfv.fit_transform(train_set.data)                # 训练，转换并返回
print(train_tfv.shape)                                       # 查看高维数组的形状（子维度数行，个数列）
print(train_tfv.toarray())                                   # 把坐标转成高维数组(ndarray类型)
# 此处相当于有2854个词袋，对这些词袋进行TfidfVectorizer进行特征提取，
# 得到最具典型的一些单词，这些单词的个数有31057个，故而得到(2854, 31057)矩阵
# 矩阵中的元素表示这个单词在该词袋中出现的tf-idf权重，值越大，表示该单词越重要。

(2854, 31057)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
# 构建模型（分类模型），训练特征
from sklearn.naive_bayes import MultinomialNB    # 机器学习工具包下的朴素贝叶斯模块中多项式分类模块
mulNB=MultinomialNB()                                        # 初始化一个多项式分布分类处理器  三个参数
# alpha加法(拉普拉斯/李德斯通)平滑参数，默认为1，0表示无平滑   fit_prior是否使用先验概率   class_prior类的先验概率。如果指定，则不根据数据调整先验
mulNB.fit(train_tfv,train_set.target)                        # 训练数据  特征 标签


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
# 在测试集test_set上进行测试
from sklearn import metrics                         # 模型评估模块
test_tfv=tfv.transform(test_set.data)                        # 将测试数据进行特征提取 训练，转换并返回
print(test_tfv.shape)                                        # 查看高维数组的形状（子维度数行，个数列）
# 预测
y_pre_test=mulNB.predict(test_tfv)                           # 预测数据
print(y_pre_test)                                            # 预测值
print(test_set.target)                                       # 真实值

F1_score=metrics.f1_score(test_set.target,y_pre_test, average='micro')  # 计算f1值 预测与平均对比  
# average平均值：macro:宏平均，micro：微平均（具体见常见名词）
print(F1_score)


(1899, 31057)
[2 1 2 ... 3 3 4]
[2 1 2 ... 3 3 4]
0.7772511848341231


In [14]:
# 分类模型报告
from sklearn.metrics import classification_report           # 模型评估模块中的分类报告模块
print(classification_report(y_pre_test,test_set.target))   # 测试值与真实值对比计算出分类准确率


              precision    recall  f1-score   support

           0       0.22      0.95      0.35        73
           1       0.92      0.89      0.90       401
           2       0.87      0.94      0.90       371
           3       0.80      0.95      0.87       333
           4       0.97      0.54      0.69       721

    accuracy                           0.78      1899
   macro avg       0.76      0.85      0.74      1899
weighted avg       0.88      0.78      0.80      1899

